In [5]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

from lightgbm import LGBMClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

## Agrupando datasets de treino

In [20]:
#Importando dados de treino
application = pd.read_csv('../base_de_dados/application_train_transformed.csv')
POS_CASH_balance = pd.read_csv('../base_de_dados/POS_CASH_balance_filled.csv')
insta_pay = pd.read_csv('../base_de_dados/installments_payments_filled.csv')
bureau = pd.read_csv('../base_de_dados/bureau_filled.csv')
features_credit = pd.read_csv('../base_de_dados/features_credito.csv')
features_appl = pd.read_csv('../base_de_dados/features_aplicacao.csv')

In [21]:
#Juntando datasets tratados
application = application.merge(bureau, on='SK_ID_CURR', how='left')
application = application.merge(POS_CASH_balance, on='SK_ID_CURR', how='left')
application = application.merge(insta_pay, on='SK_ID_CURR', how='left')
application = application.merge(features_credit, on='SK_ID_CURR', how='left')
application = application.merge(features_appl, on='SK_ID_CURR', how='left')

## Agrupando df de teste

In [22]:
application_teste = pd.read_csv('../base_de_dados/tuning_application_test_student.csv')

In [23]:
application_teste = application_teste.merge(bureau, on='SK_ID_CURR', how='left')
application_teste = application_teste.merge(POS_CASH_balance, on='SK_ID_CURR', how='left')
application_teste = application_teste.merge(insta_pay, on='SK_ID_CURR', how='left')
application_teste = application_teste.merge(features_credit, on='SK_ID_CURR', how='left')
application_teste = application_teste.merge(features_appl, on='SK_ID_CURR', how='left')

In [24]:
import re
df = application.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df_teste = application_teste.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [25]:
df.drop(['SK_ID_BUREAU'], axis=1, inplace=True)

In [26]:
application_teste.drop(['SK_ID_BUREAU'], axis=1, inplace=True)

In [32]:
application.shape

(196806, 327)

In [33]:
application_teste.shape

(49202, 327)

In [34]:
colunas_faltantes = application.columns[~pd.Series(application.columns).isin(application_teste.columns)]

if len(colunas_faltantes) > 0:
    application_teste[colunas_faltantes] = np.zeros((application_teste.shape[0], len(colunas_faltantes)))
    
colunas_faltantes

Index([], dtype='object')

In [41]:
application_teste.drop('TARGET', axis=1, inplace=True)

In [43]:
x_train = df.drop(['SK_ID_CURR', 'TARGET'], axis=1)
y_train = df['TARGET']

### Feacture Selection com Arvores

In [37]:
y_train.value_counts()

0    180764
1     16042
Name: TARGET, dtype: int64

In [44]:
sfm_lgbm = SelectFromModel(LGBMClassifier(), max_features=100)
sfm_lgbm.fit(x_train, y_train)

SelectFromModel(estimator=LGBMClassifier(), max_features=100)

In [45]:
cols = x_train.columns[sfm_lgbm.get_support()]
cols

Index(['AMT_INCOME_TOTAL', 'AMT_CREDIT_x', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'REGION_RATING_CLIENT_W_CITY',
       'REG_CITY_NOT_LIVE_CITY', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
       'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BEGINEXPLUATATION_MODE',
       'TOTALAREA_MODE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE',
       'FLAG_DOCUMENT_3', 'AMT_REQ_CREDIT_BUREAU_QRT', '0', '15', '16', '19',
       '20', '32', '36', '37', '59', 'Unnamed0_x', 'CREDIT_ACTIVE',
       'DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE', 'AMT_CREDIT_SUM',
       'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_OVERDUE', 'DAYS_CREDIT_UPDATE',
       'CREDIT_TYPE_3', 'CREDIT_TYPE_10', 'CREDIT_TYPE_12', 'valor_pago_x',
       'SK_ID_PREV', 'MONTHS_BALANCE', 'CNT_INSTALMENT',
       'CNT_INSTALMENT_FUTURE', 'SK_DPD', 'SK_DPD_DEF', 'parcelas_faltantes',
       'Unnamed0_y', 'NUM_INSTALMENT_VERSION', 'DAYS_ENTRY_PAYMEN

In [87]:
bscv = BayesSearchCV(LGBMClassifier(), 
                     search_spaces={
                        'max_depth':Integer(1, 100),
                        'n_estimators': Integer(30, 500, 1),
                        'colsample_bytree ': Real(0.1, 1),
                        'subsample': Real(0.01, 1),
                        'learning_rate': Real(0.01, 1),
                        'reg_alpha': Real(0.01, 1),
                        'reg_lambda': Real(0.01, 1),
                        'n_jobs': Integer(1, 5)
                     },
                     n_iter=500,
                     cv=3,
                     scoring='roc_auc')

bscv.fit(x_train, y_train)

[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.7285490382184888
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.7285490382184888
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.7285490382184888
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.8753473217013494
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.8753473217013494
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: 

[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 1.0
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 1.0
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.7461887774830919
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.7461887774830919
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.7461887774830919
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM

[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.968768428181029
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.968768428181029
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.968768428181029
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.11578644202039173
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.11578644202039173
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: c

[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.8901139398595938
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.13765234996256379
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.13765234996256379
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.13765234996256379
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 1.0
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_by

[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.1
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.1
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.6867286310331283
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.6867286310331283
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.6867286310331283
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM

...

[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.8766205397698287
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.5631338232213844
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.5631338232213844
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.5631338232213844
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 1.0
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytre

[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.19988640051846357
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.19988640051846357
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.7983388015830555
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.7983388015830555
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.7983388015830555
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value

[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.9746632230688372
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.9746632230688372
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.9746632230688372
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.1
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.1
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM

[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.5212304795502934
[LightGBM] [Warning] colsample_bytree is set=1.0, colsample_bytree= will be ignored. Current value: colsample_bytree=1.0
[LightGBM] [Warning] Unknown parameter: 0.20922137996135776


LightGBMError: bad allocation

In [ ]:
bscv.best_score_

In [ ]:
bscv.best_estimator_

In [ ]:
bscv.best_params_

In [57]:
x_train.shape

(199691, 336)

In [58]:
x_test.shape

(47450, 338)

In [66]:
# Mantém no x_train apenas as colunas iguais ao teste
cols_iguais = pd.Series(cols)[pd.Series(cols).isin(application_teste.columns)].values

x_train = x_train[cols_iguais]
x_test = application_teste[cols_iguais]

In [65]:
lgbm = LGBMClassifier(n_estimators=500, subsample=0.3, reg_alpha=1, reg_lambda=1, 
                     learning_rate = 0.03, max_depth=20, n_jobs=2, class_weight={0:1, 1:2},
                     colsample_bytree=1)
lgbm.fit(x_train, y_train)

LGBMClassifier(class_weight={0: 1, 1: 2}, colsample_bytree=1,
               learning_rate=0.03, max_depth=20, n_estimators=500, n_jobs=2,
               reg_alpha=1, reg_lambda=1, subsample=0.3)

In [67]:
y_pred_proba = lgbm.predict_proba(x_test)[:, 1]

In [73]:
indice = application_teste[['SK_ID_CURR']].copy()

resultado = pd.concat([indice, pd.DataFrame(y_pred_proba)], axis=1)

resultado.to_csv('../resultado/resultados.csv')